# Car Object Detection with YOLOv11n from `ultralytics`

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if not filename.endswith(".jpg"):
            print(os.path.join(dirname, filename))

In [ ]:
train_data_path = "/kaggle/input/car-object-detection/data/training_images"
test_data_path = "/kaggle/input/car-object-detection/data/testing_images"
train_bboxes_csv_path = "/kaggle/input/car-object-detection/data/train_solution_bounding_boxes (1).csv"

## Importing Dependencies

In [ ]:
import pandas as pd
import numpy as np
import cv2
import shutil
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import torch
import warnings

warnings.filterwarnings("ignore")

try:
    from ultralytics import YOLO
except:
    print("Installing `ultralytics`...")
    !pip install ultralytics
    from ultralytics import YOLO

## Organizing Directories For YOLO Model Training

In [ ]:
bboxes = pd.read_csv(train_bboxes_csv_path)
len(os.listdir(train_data_path)), bboxes.shape 

In [ ]:
# Arranging data into correct directories for training YOLO model
data_path = os.path.join(os.getcwd(), "data")
images_path = os.path.join(data_path, "images")
labels_path = os.path.join(data_path, "labels")

os.makedirs(images_path, exist_ok=True)
os.makedirs(labels_path, exist_ok=True)

In [ ]:
class ImageReadError(Exception):
    """Custom exception for image read errors."""
    def __init__(self, message="Failed to read the image. The file might be missing or corrupted."):
        self.message = message
        super().__init__(self.message)

In [ ]:
def convert_bbox_to_YOLO_format(image_path, bbox):
    try:
        image = cv2.imread(image_path)
        
        if image is None:
            raise ImageReadError(f"Failed to read image: {image_path}")

        (h, w) = image.shape[:2]
        xmin, ymin, xmax, ymax = bbox
        x_cen = (xmax + xmin) / 2. / w
        y_cen = (ymax + ymin) / 2. / h
        box_w = (xmax - xmin) / w
        box_h = (ymax - ymin) / h

        return (x_cen, y_cen, box_w, box_h)
        
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return

In [ ]:
for i in range(bboxes.shape[0]):
    img_path, bbox = bboxes.loc[i, "image"], bboxes.iloc[i, 1:].values
    full_img_path = os.path.join(train_data_path, img_path)
    YOLO_format_bbox = convert_bbox_to_YOLO_format(full_img_path, bbox)
    YOLO_format_bbox = np.array(YOLO_format_bbox).astype(str)

    with open(os.path.join(labels_path, f"{img_path.replace('.jpg', '')}.txt"), "w+") as f:
        f.write("0 " + " ".join(YOLO_format_bbox) + "\n")

    shutil.copy(full_img_path, os.path.join(images_path, img_path))

### Creating `yaml` file for YOLO model training

In [ ]:
with open(os.path.join(os.getcwd(), "dataset_custom.yaml"), "w") as f:
    f.write(f"path: {data_path}\n")
    f.write("train: images\n")
    f.write("val: images\n\n")
    f.write("nc: 1\n\n")
    f.write("names: ['car']\n")

## Displaying some images with annotations

In [ ]:
def display_image_with_bbox(image_path, bboxes):
    try:
        image = cv2.imread(image_path)

        if image is None:
            raise ImageReadError(f"Failed to read image: {image_path}")
        else:            
            for bbox in bboxes:
                x1, y1, x2, y2 = map(int, bbox)
                # Draw rectanlge:  (start_point, end_point, color, thickness)
                cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # (0, 255, 0) - Green color, thickness of 2

            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # Dispaly an image 
            plt.imshow(image_rgb)
            plt.axis("off")
            plt.title("Image with detected Car (if present)")

    except Exception as e:
        print(f"Error displaying image {image_path}: {e}")

In [ ]:
import random

plt.figure(figsize=(25, 5))

for i in range(5):
    plt.subplot(1, 5, i + 1)
    idx = random.randint(0, bboxes.shape[0] - 1)
    img_path, bbox = bboxes.loc[idx, "image"], tuple(bboxes.iloc[idx, 1:].values)
    img_path = os.path.join(train_data_path, img_path)

    display_image_with_bbox(img_path, [bbox])

## YOLO Model Training from `ultralytics`

In [ ]:
dataset_yaml_path = os.path.join(os.getcwd(), "dataset_custom.yaml")

pretrained_weights = "yolo11n.pt"  # using YOLOv11n

epochs = 30
batch_size = 16
img_size = 640 
device = "0" if torch.cuda.is_available() else "cpu"

In [ ]:
model = YOLO("yolo11n.pt")

model.train(
    data=dataset_yaml_path,
    epochs=epochs,
    batch=batch_size,
    imgsz=img_size,
    device=device,
    verbose=False
)

In [ ]:
metrics = model.val()
metrics_dict = metrics.results_dict

mAP_50 = metrics_dict['metrics/mAP50(B)']  # mAP at IoU=0.5
mAP_50_95 = metrics_dict['metrics/mAP50-95(B)']

print(f"\nmAP@0.5: {mAP_50}")
print(f"\nmAP@0.5:0.95: {mAP_50_95}")

precision = metrics_dict['metrics/precision(B)']
recall = metrics_dict['metrics/recall(B)']
fitness = metrics_dict['fitness']

print(f"\nPrecision: {precision}")
print(f"\nRecall: {recall}")
print(f"\nFitness: {fitness}")

### Model Saving

In [ ]:
trained_model_dir = os.path.join(os.getcwd(), "model")
trained_model_path = os.path.join(trained_model_dir, "car_detection_yolo.pt")

os.makedirs(trained_model_dir, exist_ok=True)

In [ ]:
model.save(trained_model_path)

## Trying Model Predictions on a Test Image

In [ ]:
idx = random.randint(0, len(os.listdir(test_data_path)) - 1)
image_name = os.listdir(test_data_path)[idx]
image_path = os.path.join(test_data_path, image_name)

img = Image.open(image_path)

results = model(img)

for result in results:
    boxes = result.boxes 
    bboxes = []

    for box in boxes:
        box = box.cpu()
        xyxy = box.xyxy.numpy()
        
        bboxes.append(tuple(xyxy[0]))
        
        confidence = box.conf.numpy()

        print(f"Bounding Box: {xyxy}")
        print(f"Confidence: {confidence}")

    display_image_with_bbox(image_path, bboxes if len(bboxes) > 0 else None)